## Ramayan Query Explorer

In [104]:
pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


#### Load story pdf

In [105]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("story_long.pdf")
docs = loader.load()
docs

[Document(page_content='', metadata={'source': 'story_long.pdf', 'page': 0}),
 Document(page_content='', metadata={'source': 'story_long.pdf', 'page': 1}),
 Document(page_content='Table of Contents\n \nAbout the Author\nTitle Page\nCopyright Page\nIntroduction\nDedication\n \nChapter 1 - RAMA’S INITIATION\nChapter 2 - THE WEDDING\nChapter 3 - TWO PROMISES REVIVED\nChapter 4 - ENCOUNTERS IN EXILE\nChapter 5 - THE GRAND TORMENTOR\nChapter 6 - VALI\nChapter 7 - WHEN THE RAINS CEASE\nChapter 8 - MEMENTO FROM RAMA\nChapter 9 - RAVANA IN COUNCIL\nChapter 10 - ACROSS THE OCEAN\nChapter 11 - THE SIEGE OF LANKA\nChapter 12 - RAMA AND RAVANA IN BATTLE\nChapter 13 - INTERLUDE\nChapter 14 - THE CORONATION\n \nEpilogue', metadata={'source': 'story_long.pdf', 'page': 2}),
 Document(page_content='Glossary', metadata={'source': 'story_long.pdf', 'page': 3}),
 Document(page_content='THE RAMAYANA\nR. K. NARAYAN was born on October 10, 1906, in Madras,\nSouth India, and educated there and at Maharaja’s C

#### Split document into chunks

In [106]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(page_content='Table of Contents\n \nAbout the Author\nTitle Page\nCopyright Page\nIntroduction\nDedication\n \nChapter 1 - RAMA’S INITIATION\nChapter 2 - THE WEDDING\nChapter 3 - TWO PROMISES REVIVED\nChapter 4 - ENCOUNTERS IN EXILE\nChapter 5 - THE GRAND TORMENTOR\nChapter 6 - VALI\nChapter 7 - WHEN THE RAINS CEASE\nChapter 8 - MEMENTO FROM RAMA\nChapter 9 - RAVANA IN COUNCIL\nChapter 10 - ACROSS THE OCEAN\nChapter 11 - THE SIEGE OF LANKA\nChapter 12 - RAMA AND RAVANA IN BATTLE\nChapter 13 - INTERLUDE\nChapter 14 - THE CORONATION\n \nEpilogue', metadata={'source': 'story_long.pdf', 'page': 2}),
 Document(page_content='Glossary', metadata={'source': 'story_long.pdf', 'page': 3}),
 Document(page_content='THE RAMAYANA\nR. K. NARAYAN was born on October 10, 1906, in Madras,\nSouth India, and educated there and at Maharaja’s College\nin Mysore. His first novel, \nSwami and Friends\n (1935), and its\nsuccessor, \nThe Bachelor of Arts\n (1937), are both set in the\nfictional territ

In [107]:
documents=text_splitter.split_documents(docs)
documents

[Document(page_content='Table of Contents\n \nAbout the Author\nTitle Page\nCopyright Page\nIntroduction\nDedication\n \nChapter 1 - RAMA’S INITIATION\nChapter 2 - THE WEDDING\nChapter 3 - TWO PROMISES REVIVED\nChapter 4 - ENCOUNTERS IN EXILE\nChapter 5 - THE GRAND TORMENTOR\nChapter 6 - VALI\nChapter 7 - WHEN THE RAINS CEASE\nChapter 8 - MEMENTO FROM RAMA\nChapter 9 - RAVANA IN COUNCIL\nChapter 10 - ACROSS THE OCEAN\nChapter 11 - THE SIEGE OF LANKA\nChapter 12 - RAMA AND RAVANA IN BATTLE\nChapter 13 - INTERLUDE\nChapter 14 - THE CORONATION\n \nEpilogue', metadata={'source': 'story_long.pdf', 'page': 2}),
 Document(page_content='Glossary', metadata={'source': 'story_long.pdf', 'page': 3}),
 Document(page_content='THE RAMAYANA\nR. K. NARAYAN was born on October 10, 1906, in Madras,\nSouth India, and educated there and at Maharaja’s College\nin Mysore. His first novel, \nSwami and Friends\n (1935), and its\nsuccessor, \nThe Bachelor of Arts\n (1937), are both set in the\nfictional territ

#### No.of chunks

In [108]:
len(documents)

507

In [11]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [109]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

#### Creating embeddings for the chunks and storing them in a vectorstore

In [ ]:
#no need to run every time
db=FAISS.from_documents(documents,OllamaEmbeddings())

In [82]:
#save in local
db.save_local('/Users/anu/Documents/NLP_PACKAGE')

In [111]:
#load vector indices
db=FAISS.load_local('/Users/anu/Documents/NLP_PACKAGE',OllamaEmbeddings())

In [112]:
db

#### Verify vector retrieval from DB using similarity_search

In [143]:
query='Ravan kidnapped Sita'
result=db.similarity_search(query)
result[0].page_content

'misery.”\nA few encouraged this idea, but one of his councillors\nadvised, “Don’t defeat your own purpose and integrity by\nkilling a woman. Let your anger scorch Rama and his\nbrother. Gather all your armies and go and vanquish Rama'

#### Verify vector retrieval from DB using similarity_search_by_vector

In [114]:
embedding_vector = OllamaEmbeddings().embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

and Lakshmana, you know you can, and then take Sita. Put
on your blessed armour and go forth.”


#### Generator- LLM

In [115]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

#### Define prompt template

In [130]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
You are a chatbot designed to answer questions about the Hindu Mythological epic Ramayana. 
Answer the question precisely based on the context and your knowledge. Maintain a neutral tone.
<context>
{context}
</context>
Question: {input}""")

#### QandA chain

In [131]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [132]:
"""
Retrievers: An interface that returns documents given
an unstructured query. 
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13f5f9f10>)

#### RAG chain

In [133]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

#### Test out few queries

In [128]:
response=retrieval_chain.invoke({"input":"Who is Ram"})
print(response['answer'])

Ram is the hero of the Hindu Mythological epic Ramayana. He is the prince of Ayodhya and the husband of Sita. According to the story, Ram was born to King Dasaratha and Queen Kausalya, and he was the eldest of three brothers. He is known for his bravery, virtue, and righteousness, and is considered a model of moral excellence in Hindu culture.


In [134]:
response=retrieval_chain.invoke({"input":"Who are the main characters in Ramayana?"})
print(response['answer'])

Thank you for providing the context. The main characters in the Ramayana are:

1. Rama: The protagonist of the story, Rama is the eldest son of King Dasharatha and Queen Kausalya. He is known for his virtues such as truthfulness, courage, and selflessness.
2. Ravana: The main antagonist of the story, Ravana is a powerful demon king who abducts Rama's wife, Sita, and is determined to stop Rama from rescuing her.
3. Sita: The wife of Rama and the object of Ravana's abduction, Sita is known for her devotion and loyalty to her husband.
4. Lakshmana: Rama's younger brother, Lakshmana is a skilled warrior and a loyal companion to Rama on his quest to rescue Sita.
5. Hanuman: A monkey god and a loyal servant of Rama, Hanuman plays a significant role in the story as a messenger and a warrior.
6. King Dasharatha: The father of Rama and Lakshmana, King Dasharatha is a just and fair ruler who is torn between his duty to his sons and his love for his wife, Kausalya.
7. Queen Kausalya: The mother o

In [135]:
response=retrieval_chain.invoke({"input":"members of Rama's royal family"})
print(response['answer'])

In the context you provided, the members of Rama's royal family are:

1. Rama: The eldest son of King Dasaratha and the protagonist of the Ramayana. He is exiled to the forest for 14 years after his stepmother Kaikeyi fulfills her wish to send him away.
2. Lakshmana: Rama's younger brother and companion during his exile. He is known for his loyalty and bravery in protecting Rama and their mother, Queen Kausalya.
3. Dasaratha: The King of Ayodhya and Rama and Lakshmana's father. He is mentioned in the context as being childless and praying to the gods for an issue, which leads to the exile of Rama.
4. Kaikeyi: Rama's stepmother and the instigator of his exile. She is known for her cunning and ambition, and her desire to see Rama away from the throne.
5. Sumithra: Rama's step-mother and Dasaratha's second wife. She bewailed Rama's exile and tried to stop him from leaving, but ultimately accepted his decision.
6. Ravana: The King of Lanka and the main antagonist of the Ramayana. He is des

In [136]:
response=retrieval_chain.invoke({"input":"Name the four sons of king dasaratha"})
print(response['answer'])

According to the context provided, the four sons of King Dasaratha are:

1. Rama
2. Bharata
3. Lakshmana
4. Shatrughna


In [137]:
response=retrieval_chain.invoke({"input":"How long was Rama's exile"})
print(response['answer'])

According to the Ramayana, Rama's exile lasted for 14 years. After being banished to the forest by his father, King Dasaratha, Rama lived in exile with his wife Sita and brother Lakshmana for this duration of time.


In [138]:
response=retrieval_chain.invoke({"input":"Name of the hill that Hanuman brought to save Lakshman?"})
print(response['answer'])

According to the context provided, Hanuman brought a hill named "Sankha Parvata" to save Lakshmana from the clutches of Ravana.


In [139]:
response=retrieval_chain.invoke({"input":"Ravana's wife name"})
print(response['answer'])

Based on the context provided, Ravana's wife's name is Mandodari.


In [149]:
response=retrieval_chain.invoke({"input":"Who is Sugreeva"})
print(response['answer'])

Sugreeva is the king of the Vanaras, a race of semi-divine beings in Hindu mythology. He is mentioned in the context you provided as the one who installed Rama's sandals on the throne and ruled the country as a regent. Sugreeva is also known as Vali's younger brother and is considered to be a powerful warrior and leader.


In [30]:
import pickle
with open('retriever.pkl','wb') as f:
    pickle.dump(retriever,f)